# Query Wikidata

In [ ]:
import pandas as pd

from pandas.io.json import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON

In [ ]:
def query_wikidata(sparql_query, sparql_service_url):
    """
    Query the endpoint with the given query string and return the results as a pandas Dataframe.
    """
    # create the connection to the endpoint
    # Wikidata enforces now a strict User-Agent policy, we need to specify the agent
    # See here https://www.wikidata.org/wiki/Wikidata:Project_chat/Archive/2019/07#problems_with_query_API
    # https://meta.wikimedia.org/wiki/User-Agent_policy
    # Please adapt this when running
    sparql = SPARQLWrapper(sparql_service_url, agent="https://github.com/mfingerh")  
    
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # ask for the result
    result = sparql.query().convert()
    return json_normalize(result["results"]["bindings"])

In [ ]:
# suppresses a future warning the query evokes
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# perfirn sparqle query
sparql_query = """
SELECT DISTINCT ?p ?pLabel ?affiliationTypeLabel ?w ?wLabel WHERE {
  ?p wdt:P31 wd:Q98270496;
    p:P1416 ?statement.
  ?statement ps:P1416 ?w.
  OPTIONAL { ?statement pq:P3831 ?affiliationType. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY DESC (?wLabel) 
    """
#to query another endpoint, change the URL for the service and the query
sparql_service_url = "https://query.wikidata.org/sparql"
result_table = query_wikidata(sparql_query, sparql_service_url)



# Reformating and selecting data

In [ ]:
result_table.rename(columns = {'affiliationTypeLabel.value':'Rolle', 'wLabel.value':'Einrichtung'}, inplace = True)

In [ ]:
crosstab = pd.crosstab(
    index=result_table['Einrichtung'],
    columns=result_table['Rolle'],
)

In [ ]:
# cecking everything worked
crosstab.head()

In [ ]:
# list of columns to be considered for calculating sum, can be changed to consider eg. only applicants and co-applicants
columns = ['applicant', 'co-applicant', 'participant']

In [ ]:
# calculating sums of roles to enable ordering
crosstab['sum'] = crosstab[columns].sum(axis=1, numeric_only=True)

In [ ]:
# ordering by sum from most to least consortia
crosstab = crosstab.sort_values(by=['sum'], ascending=False)

In [ ]:
# filtering by sum to have a threshold for plotted entries
crosstab = crosstab[crosstab[columns].sum(axis=1, numeric_only=True) > 4]

In [ ]:
# removing sum column as it's not needed for plotting
crosstab = crosstab.drop(['sum'], axis=1)

In [ ]:
crosstab.head()

# Barplot

In [ ]:
# plotting
crosstab.plot.bar(stacked=True, figsize=(30,34), fontsize = 20, position = (0.3),width=0.9, title = 'Beteiligung an der NFDI in nach der dritten Ausschreibungsrunde')